In [1]:
import pandas as pd
import torch
import numpy as np

/usr/lib/python3/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# return a list of tuples [(a, b,c)]
def get_samples(lines):
    samples = []
    for line in lines:
        # take advantage of the coloring thing
        if line.startswith('\x1b[92m'):
            clean_line = line.replace('\x1b[92m', '').replace('\x1b[0m', '').strip().replace(',', '')
            a = clean_line.split('=')[0]
            c = clean_line.split('=')[1]
            a, b = a.split('+')
            samples.append((int(a), int(b), c.strip()))
    return samples

# return a list of tuples [(a, b,c)] for few-shot results
def get_samples_fewshot(lines):
    samples = []
    for line in lines:
        # take advantage of the coloring thing
        if line.startswith('\x1b[92m'):
            clean_line = line.replace('\x1b[92m', '').replace('\x1b[0m', '').strip().replace(',', '')
            a = clean_line.split('=')[0]
            c = clean_line.split('=')[1]
            a, b = a.split('+')
            samples.append((int(a), int(b), c.strip()))
    return samples

# return a list of tuples [(a, b,c)] for few-shot results
def get_samples_fewshot(lines, num_fewshot=5):
    samples = []
    for line in lines:
        # take advantage of the coloring thing
        if line.startswith('\x1b[92m'):
            clean_line = line.replace('\x1b[92m', '').replace('\x1b[0m', '').strip().replace(',', '').split(';')[num_fewshot:]
            a = clean_line[0].strip().split('=')[0]
            c = clean_line[0].strip().split('=')[1] + ';' + ';'.join(clean_line[1:])
            a, b = a.split('+')
            samples.append((int(a), int(b), c.strip()))
    return samples

models = ['llama-7b', 'mpt-7b', 'mpt-7b-instruct']
samples_dict = {}
for model in models:
    for suffix in ['-fewshot-results-0-temp-without-spaces.txt']:
        filename = model + suffix
        with open(filename, 'r') as f:
            lines = f.readlines()
            print(f'Processing: {filename}')
            samples = get_samples_fewshot(lines, num_fewshot=5)
            key = (model + '-without-spaces') if 'without-spaces' in suffix else (model + '-with-spaces')
            samples_dict[key] = samples

models = ['llama-7b', 'mpt-7b', 'mpt-7b-instruct']
samples_dict = {}
for model in models:
    for suffix in ['-results-0-temp-with-spaces.txt', '-results-0-temp-without-spaces.txt', '-fewshot-results-0-temp-without-spaces.txt',
                   '-10-fewshot-results-0-temp-without-spaces.txt',]:
        filename = model + suffix
        with open(filename, 'r') as f:
            lines = f.readlines()
            print(f'Processing: {filename}')
            if 'fewshot' in suffix:
                num_fewshot = 5
                if '10' in suffix:
                    num_fewshot = 10
                samples = get_samples_fewshot(lines, num_fewshot=num_fewshot)
                key = (model + f'-{num_fewshot}-fewshot-without-spaces') if 'without-spaces' in suffix else (model + f'-{num_fewshot}-fewshot-with-spaces')
            else:
                samples = get_samples(lines)
                key = (model + '-without-spaces') if 'without-spaces' in suffix else (model + '-with-spaces')
            samples_dict[key] = samples

Processing: llama-7b-fewshot-results-0-temp-without-spaces.txt
Processing: mpt-7b-fewshot-results-0-temp-without-spaces.txt
Processing: mpt-7b-instruct-fewshot-results-0-temp-without-spaces.txt
Processing: llama-7b-results-0-temp-with-spaces.txt
Processing: llama-7b-results-0-temp-without-spaces.txt
Processing: llama-7b-fewshot-results-0-temp-without-spaces.txt
Processing: mpt-7b-results-0-temp-with-spaces.txt
Processing: mpt-7b-results-0-temp-without-spaces.txt
Processing: mpt-7b-fewshot-results-0-temp-without-spaces.txt
Processing: mpt-7b-instruct-results-0-temp-with-spaces.txt
Processing: mpt-7b-instruct-results-0-temp-without-spaces.txt
Processing: mpt-7b-instruct-fewshot-results-0-temp-without-spaces.txt


In [3]:
# evaluate samples
def evaluate_samples(samples, debug_mode=False):
    correct = 0
    total = len(samples)
    correct_samples = []
    incorrect_samples = []
    for (a, b, c) in samples:
        true_c = str(a + b)
        if c.startswith(true_c):
            correct += 1
            correct_samples.append((a, b, c))
            if debug_mode:
                print(f"Correct! {a} + {b} = {c} = {true_c}")
        else:
            incorrect_samples.append((a, b, c))
            if debug_mode:
                print(f"Wrong! {a} + {b} = {c} != {true_c}")
    return 100.0 * correct / total, correct_samples, incorrect_samples

In [4]:
results_dict = {}
correct_samples_dict = {}
incorrect_samples_dict = {}
for key in samples_dict.keys():
    results_dict[key], correct_samples_dict[key], incorrect_samples_dict[key] = evaluate_samples(samples_dict[key])

In [5]:
results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['Accuracy'])
results_df

,Accuracy
llama-7b-with-spaces,80.2
llama-7b-without-spaces,75.7
llama-7b-fewshot-without-spaces,77.5
mpt-7b-with-spaces,6.0
mpt-7b-without-spaces,6.0
mpt-7b-fewshot-without-spaces,41.9
mpt-7b-instruct-with-spaces,7.3
mpt-7b-instruct-without-spaces,7.3
mpt-7b-instruct-fewshot-without-spaces,37.1


In [6]:
def print_pretty_results(samples, correct_answers=True):
    CRED = '\033[91m'
    CEND = '\033[0m'
    CGREEN = '\033[92m'
    CBLUE = '\033[94m'
    for (a, b, c) in samples:
        true_c = str(a + b)
        if correct_answers:
            print(f'{a} + {b} = {CBLUE} {c} {CEND} = {CGREEN}' + f'{true_c}{CEND}')
        else:
            print(f'{a} + {b} = {CBLUE} {c} {CEND} = {CRED}' + f'{true_c}{CEND}')
        

In [7]:
print_pretty_results(correct_samples_dict['mpt-7b-without-spaces'][:10], correct_answers=True)

131 + 785 =  916. The number 916 is a composite  = 916
211 + 11 =  222. The number of the beast.  = 222
33 + 62 =  95. The number of days until the end of  = 95
692 + 652 =  1344. The number 1344 is the sum  = 1344
315 + 145 =  460. The number of days in a year.  = 460
598 + 119 =  717. The sum of the digits of the number  = 717
1000 + 239 =  1239. The number 1239 is a composite  = 1239
536 + 235 =  771. The number 771 is the sum of  = 771
140 + 145 =  285. The number of days in the year.  = 285
127 + 107 =  234. The number of days until the next World  = 234


In [8]:
print_pretty_results(incorrect_samples_dict['mpt-7b-without-spaces'][:10], correct_answers=False)

813 + 124 =  1827. The number of days in  = 937
65 + 588 =  719. The number 719 is a composite  = 653
26 + 512 =  1313. The number 1313 is a prime  = 538
622 + 197 =  739. 739 is the sum of the  = 819
612 + 493 =  719. The number 719 is a composite  = 1105
733 + 190 =  943. The number 943 is a composite  = 923
787 + 29 =  826. The number 826 is a pal  = 816
891 + 154 =  1916. The number of days in  = 1045
178 + 567 =  789. The sum of the digits of the  = 745
622 + 35 =  669. The number 669 is a composite  = 657


In [9]:
print_pretty_results(correct_samples_dict['mpt-7b-instruct-without-spaces'][:10], correct_answers=True)

211 + 11 =  222. The number of the beast.  = 222
174 + 154 =  328. The sum of the first two numbers is  = 328
33 + 62 =  95. The number of days until the next World  = 95
103 + 148 =  251  = 251
844 + 579 =  1423  = 1423
731 + 84 =  815  = 815
536 + 235 =  771. The sum of the digits in the number  = 771
846 + 73 =  919. The sum of the digits of the  = 919
362 + 316 =  678. The sum of the digits of the number  = 678
342 + 141 =  483. The sum of the digits of the number  = 483


In [10]:
print_pretty_results(incorrect_samples_dict['mpt-7b-instruct-without-spaces'][:50], correct_answers=False)

813 + 124 =  1837  = 937
65 + 588 =  719  = 653
26 + 512 =  1337. I love you.  = 538
622 + 197 =  829. The sum of the digits of the  = 819
612 + 493 =  1215  = 1105
733 + 190 =  943. The sum of the digits of the  = 923
787 + 29 =  826. The sum of the digits in the  = 816
891 + 154 =  1965. The sum of the digits  = 1045
178 + 567 =  749 so the answer is 749.  = 745
622 + 35 =  629. The sum of the digits of the  = 657
146 + 954 =  1965. The sum of the digits  = 1100
694 + 828 =  1826  = 1522
984 + 318 =  1224  = 1302
843 + 635 =  1758  = 1478
768 + 171 =  929  = 939
944 + 326 =  1269  = 1270
392 + 655 =  1819  = 1047
403 + 72 =  555. The number of days in the year.  = 475
653 + 480 =  1023. The number 1023 is called a  = 1133
970 + 294 =  1815  = 1264
131 + 785 =  1026. The sum of the digits of the  = 916
210 + 461 =  621. The sum of the digits of the  = 671
15 + 749 =  15+749  = 764
388 + 542 =  746. The sum of the digits of the  = 930
976 + 682 =  1568 which is a prime number.  = 1658


In [11]:
print_pretty_results(correct_samples_dict['mpt-7b-instruct-fewshot-without-spaces'][:10], correct_answers=True)

690 + 230 =  920.;  = 920
229 + 994 =  1223; 965+965=18  = 1223
15 + 88 =  103; 8+8=16.  = 103
966 + 314 =  1280; 965+965=19  = 1280
593 + 699 =  1292.;  = 1292
202 + 854 =  1056.;  = 1056
751 + 802 =  1553; 965+965=19  = 1553
948 + 510 =  1458.;  = 1458
759 + 916 =  1675; 965+965=19  = 1675
707 + 541 =  1248.;  = 1248


In [12]:
print_pretty_results(incorrect_samples_dict['mpt-7b-instruct-fewshot-without-spaces'][:10], correct_answers=False)

264 + 620 =  836.;  = 884
826 + 763 =  1789; 765+828=17  = 1589
711 + 813 =  1414.;  = 1524
17 + 372 =  385; 9+9=18; 8+  = 389
993 + 560 =  1743; 865+928=17  = 1553
744 + 153 =  977; 965+965=18  = 897
298 + 139 =  447; 765+914=1769  = 437
333 + 837 =  1150.;  = 1170
695 + 137 =  834; 865+928=17  = 832
397 + 361 =  748.;  = 758
